In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("D:/DataIITM/sourcelocalization.csv")

In [3]:
data.head()

,Unnamed: 0,rec_x,rec_y,rec_z,roll_r,pitch_r,yaw_r,mag_x,mag_y,mag_z,target_x,target_y,target_z
0,0,-58.0,-58.0,-58.0,-90.0,-90.0,-90.0,7.090135e-08,7.090135e-08,-2.659524e-23,89.720577,-52.311175,3.709327
1,1,-58.0,-58.0,-58.0,-90.0,-90.0,-45.0,7.090135e-08,5.013483e-08,-5.013483e-08,62.392870,-28.577314,78.041444
2,2,-58.0,-58.0,-58.0,-90.0,-90.0,0.0,7.090135e-08,-2.659524e-23,-7.090135e-08,26.755383,45.103964,89.720577
3,3,-58.0,-58.0,-58.0,-90.0,-90.0,45.0,7.090135e-08,-5.013483e-08,-5.013483e-08,16.640779,98.782895,27.659070
4,4,-58.0,-58.0,-58.0,-90.0,-90.0,90.0,7.090135e-08,-7.090135e-08,1.791233e-23,41.651419,81.499062,-49.224610


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15625000 entries, 0 to 15624999
Data columns (total 13 columns):
 #   Column      Dtype  
---  ------      -----  
 0   Unnamed: 0  int64  
 1   rec_x       float64
 2   rec_y       float64
 3   rec_z       float64
 4   roll_r      float64
 5   pitch_r     float64
 6   yaw_r       float64
 7   mag_x       float64
 8   mag_y       float64
 9   mag_z       float64
 10  target_x    float64
 11  target_y    float64
 12  target_z    float64
dtypes: float64(12), int64(1)
memory usage: 1.5 GB


In [5]:
data.drop("Unnamed: 0" , axis =1, inplace=True)

In [6]:
data.head()

,rec_x,rec_y,rec_z,roll_r,pitch_r,yaw_r,mag_x,mag_y,mag_z,target_x,target_y,target_z
0,-58.0,-58.0,-58.0,-90.0,-90.0,-90.0,7.090135e-08,7.090135e-08,-2.659524e-23,89.720577,-52.311175,3.709327
1,-58.0,-58.0,-58.0,-90.0,-90.0,-45.0,7.090135e-08,5.013483e-08,-5.013483e-08,62.392870,-28.577314,78.041444
2,-58.0,-58.0,-58.0,-90.0,-90.0,0.0,7.090135e-08,-2.659524e-23,-7.090135e-08,26.755383,45.103964,89.720577
3,-58.0,-58.0,-58.0,-90.0,-90.0,45.0,7.090135e-08,-5.013483e-08,-5.013483e-08,16.640779,98.782895,27.659070
4,-58.0,-58.0,-58.0,-90.0,-90.0,90.0,7.090135e-08,-7.090135e-08,1.791233e-23,41.651419,81.499062,-49.224610


In [7]:
from sklearn.preprocessing import StandardScaler 
from sklearn.pipeline import Pipeline 